In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from scipy.spatial import distance
from sklearn.decomposition import PCA
import matplotlib.cm as matpltlb
import matplotlib.pyplot as plot


In [8]:
def Kmeans(X_train,N,centroids,max_iteration_number):
    """
    :type X_train: numpy.ndarray
    :type N: int
    :rtype: List[numpy.ndarray]
    """

    n = len(X_train)
    # centroids = X_train[np.random.choice(X_train.shape[0], size = N, replace= False),:]
    # print(type(centroids))
    centers_old = np.zeros(centroids.shape) # to store old centers
    previous_centroids = centroids.copy() # Store new centers

    clusters = np.zeros(n)
    distances = np.zeros((n,N))

    samecluster = False
    i = 0
    while (not samecluster) or i > max_iteration_number :
        clusternew = []
        for i in range(N):
            distances[:,i] = np.linalg.norm(X_train - centroids[i], axis=1)
        clusters = np.argmin(distances, axis = 1)
      
        
        new_centroids = previous_centroids.copy()
        for i in range(N):
            previous_centroids[i] = np.mean(X_train[clusters == i], axis=0)
            clusternew.append(X_train[clusters == i])

        samecluster = np.array_equal(previous_centroids, new_centroids)
        i+=1
    return clusternew,clusters,new_centroids

In [ ]:
# load dataset from the specified path
import numpy as np
import seaborn as sns


dt = pd.read_csv("/content/new_dataset_1.txt",delimiter="\t",header=None)                                    #file name to be changed
genes = dt.iloc[:, :]  #importing the entire dataset
gene_data = []

# k = 3
# centroid_number = [3,5,9]
# for eachcentroid in centroid_number:
#   centroids = data[eachcentroid-1]

# preprocessing and converting the dataset to a list of list by removing the first column and label column
for i in range(0,len(genes)):
  temp = []
  for j in range(0,len(genes.columns)):
    genes[j][i] = str(genes[j][i])
    temp.append(genes[j][i])
  gene_data.append(temp)
data=np.asarray(gene_data)
label = data[:,1]
data= np.asarray(data[:,2:],dtype=float)

#Getting inputs
k = int(input("Enter k value "))
# print("k = "+str(k))
max_iteration_number = int(input("Enter max iteration number: "))

#numpycentroid = data[np.random.choice(data.shape[0], size = k, replace= False),:]                                         #default centroid initialization


centroids = []                                                                                                          #to initialize the centroid value from console uncomment from line34 to line38 here
for i in range(k):
  index = int(input("Enter the IDs = "+str(i+1)))
  centroids.append(data[index-1])
  numpycentroid = np.array(centroids)

clusternew,clusters,new_centroids = Kmeans(data,k,numpycentroid,max_iteration_number)


def jaccard_and_rand(data,label,cluster_final):
  data_array = np.asarray(data, dtype = float)
  # gene_id = data_array[:,0]
  # ground_truth = data_array[:,1]
  # attributes = np.delete(data_array,np.s_[0:2],axis = 1)
  true_positive = 0
  true_negative = 0
  false_positive=0
  false_negative=0
  for i in range(len(data_array)):
    for j in range(len(data_array)):
      if label[i]==label[j]:
        if cluster_final[i]==cluster_final[j]:
          true_positive=true_positive+1
        else:
          false_negative=false_negative+1
      elif label[i]!=label[j]:
        if cluster_final[i]==cluster_final[j]:
          false_positive=false_positive+1
        else:
          true_negative=true_negative+1
  jaccard=(true_positive)/(true_positive+false_positive+false_negative)
  rand=(true_positive+true_negative)/(true_positive+true_negative+false_positive+false_negative)
  return jaccard,rand   

#performs PCA dimensionality reduction on the data when the number of attributes is greater than 2 and plots the scatter plot
def PCA_DimensionalityReduction(data,weights):
  pca_output = PCA(n_components=2).fit_transform(data)
  pca_output =np.asarray(pca_output)
  weights = np.asarray(weights)
  pca_output_df = pd.DataFrame(pca_output, columns=['Component1','Component2'], index=label)
  weights_df = pd.DataFrame(weights)
  pca_output_df['cluster_id'] = weights
  #plotting using seaborn
  lm = sns.lmplot(x='Component1', y='Component2', data=pca_output_df, fit_reg=False, hue='cluster_id')
  plt.show()

#Plots the scatter plot when the number of attributes is less than 2 
def visualization(data,weights):
  data_output =np.asarray(data)
  weights = np.asarray(weights)
  pca_output_df = pd.DataFrame(data_output, columns=['Column1','Column2'], index=label)
  weights_df = pd.DataFrame(weights)
  pca_output_df['cluster_id'] = weights
  #plotting using seaborn
  lm = sns.lmplot(x='Column1', y='Column2', data=pca_output_df, fit_reg=False, hue='cluster_id')
  plt.show()


jaccard_value,rand_index_value=jaccard_and_rand(data,label,clusters)
print("Jaccard Coefficient = ",jaccard_value)
print("Rand Index = ",rand_index_value)


#calling PCA and visualization based on the dimensionality of the dataset
num_cols = np.shape(data)[1]
if num_cols > 2:
  PCA_DimensionalityReduction(data,clusters)
else:
  visualization(data, clusters)
# print(data.shape)
# print(label.shape)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
